<a href="https://colab.research.google.com/github/SharpQuagga/Colab_Notebooks/blob/master/AgeAndGender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install OpenCV-python
!pip install pafy
!pip install youtube_dl


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import cv2
import numpy as np
import pafy
#url of the video to predict Age and gender
url = 'https://www.youtube.com/watch?v=c07IsbSNqfI&feature=youtu.be'
vPafy = pafy.new(url)
play = vPafy.getbest(preftype="mp4") 
cap = cv2.VideoCapture(play.url)
print(play.url)

https://r5---sn-5hnedn7l.googlevideo.com/videoplayback?expire=1573516946&ei=MqLJXdHHHoKu7gOis5rIBQ&ip=35.204.193.3&id=o-AG8SXKik_1Dijp2rf-IdtKMJ84XNFNDvmDgAS8VyZHS5&itag=22&source=youtube&requiressl=yes&mm=31%2C29&mn=sn-5hnedn7l%2Csn-5hne6n7l&ms=au%2Crdu&mv=m&mvi=4&pl=20&initcwndbps=2033750&mime=video%2Fmp4&ratebypass=yes&dur=1687.579&lmt=1520524663590094&mt=1573495161&fvip=5&fexp=23842630&c=WEB&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cmime%2Cratebypass%2Cdur%2Clmt&sig=ALgxI2wwRQIgAXzx6urDOo9J_HVsVLGkdjRKaMlDoeo-09Pft1lSVQICIQDTVlaKF0E0OGgBpauImQAjM9KKOofY2vKO4dIHhKIB4w%3D%3D&lsparams=mm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AHylml4wRgIhAIuivtoq90yJrF_ARjIhSLpnY3rbNtf1b-lXJWFoQLavAiEA_mQ1uUWl6_DrSqsfbTdNSa8KXc6UumnBiz_HnPPooXU%3D


In [0]:

cap.set(3, 480) #set width of the frame
cap.set(4, 640) #set height of the frame
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
gender_list = ['Male', 'Female']


def load_caffe_models():
 
    age_net = cv2.dnn.readNetFromCaffe('/content/drive/My Drive/Age and Gender/deploy_age.prototxt', '/content/drive/My Drive/Age and Gender/age_net.caffemodel')
    gender_net = cv2.dnn.readNetFromCaffe('/content/drive/My Drive/Age and Gender/deploy_gender.prototxt', '/content/drive/My Drive/Age and Gender/gender_net.caffemodel')
    return(age_net, gender_net)


def video_detector(age_net, gender_net):
    font = cv2.FONT_HERSHEY_SIMPLEX
    while True:
        ret, image = cap.read()
        
        face_cascade = cv2.CascadeClassifier('/content/drive/My Drive/Age and Gender/haarcascade_frontalface_default.xml')
        if ret:
          gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
          faces = face_cascade.detectMultiScale(gray, 1.1, 5)

          if(len(faces)>0):
              print("Found {} faces".format(str(len(faces))))
              for (x, y, w, h )in faces:
                  cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 0), 2)
                  #Get Face 
                  face_img = image[y:y+h, h:h+w].copy()
                  blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
                  #Predict Gender
                  gender_net.setInput(blob)
                  gender_preds = gender_net.forward()
                  gender = gender_list[gender_preds[0].argmax()]
                  print("Gender : " + gender)
                  #Predict Age
                  age_net.setInput(blob)
                  age_preds = age_net.forward()
                  age = age_list[age_preds[0].argmax()]
                  print("Age Range: " + age)
                  overlay_text = "%s %s" % (gender, age)
                  cv2.putText(image, overlay_text, (x, y), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
                  cv2.imshow('frame', image)  
                  #0xFF is a hexadecimal constant which is 11111111 in binary.
                  if cv2.waitKey(1) & 0xFF == ord('q'): 
                      break

if __name__ == "__main__":
    age_net, gender_net = load_caffe_models()
    video_detector(age_net, gender_net)
